In [1]:
import os
import librosa
import soundfile as sf
from librosa.core import load, stft, istft, magphase
from librosa.output import write_wav
from config import *
from concurrent.futures import ThreadPoolExecutor   
from time import time
import asyncio
import numpy as np
from multiprocessing import cpu_count

In [2]:
dirs_dev = list(os.walk('../DSD100_extracted/Dev/'))[0][1]
# dirs_test = list(os.walk('../DSD100_extracted/Test/'))[0][1]

def downsample(input_path, output_path):
    wav, _ = librosa.load(input_path, sr=SAMPLE_RATE)
    write_wav(output_path, wav, SAMPLE_RATE, norm=True)
    print(f"Saving {output_path}")

if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs_dev)):
            target_dir = '../DSD100_resized/Dev/{}/'.format(dirs_dev[i])
            os.makedirs(target_dir, exist_ok=True)                    
            pool.submit(downsample, f'../DSD100_extracted/Dev/{dirs_dev[i]}/mixture.wav', target_dir + 'mixture.wav')
            pool.submit(downsample, f'../DSD100_extracted/Dev/{dirs_dev[i]}/vocals.wav', target_dir + 'vocals.wav')
            pool.submit(downsample, f'../DSD100_extracted/Dev/{dirs_dev[i]}/accompaniment.wav', target_dir + 'accompaniment.wav')
        # for i in range(len(dirs_test)):
        #     target_dir = '../DSD100_resized/Test/{}/'.format(dirs_test[i])
        #     os.makedirs(target_dir, exist_ok=True)                    
        #     pool.submit(downsample, f'../DSD100_extracted/Test/{dirs_test[i]}/mixture.wav', target_dir + 'mixture.wav')
        #     pool.submit(downsample, f'../DSD100_extracted/Test/{dirs_test[i]}/vocals.wav', target_dir + 'vocals.wav')
        #     pool.submit(downsample, f'../DSD100_extracted/Test/{dirs_test[i]}/accompaniment.wav', target_dir + 'accompaniment.wav')


Saving ../DSD100_resized/Dev/053 - Actions - Devil's Words/accompaniment.wav
Saving ../DSD100_resized/Dev/052 - ANiMAL - Easy Tiger/vocals.wav
Saving ../DSD100_resized/Dev/054 - Actions - South Of The Water/vocals.wav
Saving ../DSD100_resized/Dev/055 - Angels In Amplifiers - I'm Alright/accompaniment.wav
Saving ../DSD100_resized/Dev/055 - Angels In Amplifiers - I'm Alright/vocals.wav
Saving ../DSD100_resized/Dev/056 - Arise - Run Run Run/mixture.wav
Saving ../DSD100_resized/Dev/052 - ANiMAL - Easy Tiger/accompaniment.wav
Saving ../DSD100_resized/Dev/056 - Arise - Run Run Run/accompaniment.wav
Saving ../DSD100_resized/Dev/051 - AM Contra - Heart Peripheral/mixture.wav
Saving ../DSD100_resized/Dev/051 - AM Contra - Heart Peripheral/vocals.wavSaving ../DSD100_resized/Dev/057 - BKS - Bulldozer/accompaniment.wavSaving ../DSD100_resized/Dev/054 - Actions - South Of The Water/mixture.wavSaving ../DSD100_resized/Dev/058 - Ben Carrigan - We'll Talk About It All Tonight/mixture.wavSaving ../DSD1

In [3]:
def load_as_mag(file):
    wav, _ = librosa.load(file, sr=None)
    spectrogram = stft(wav, n_fft=WINDOW_SIZE, hop_length=HOP_LENGTH)
    mag, _ = magphase(spectrogram)
    return mag.astype(np.float32)

def save_to_npz(base, sub, sample):
    nps = {}
    mix = load_as_mag(f'{base}/{sample}/mixture.wav')
    vocal = load_as_mag(f'{base}/{sample}/vocals.wav')
    acc = load_as_mag(f'{base}/{sample}/accompaniment.wav')
        
    mix_max = mix.max()
    mix_norm = mix / mix_max
    vocal_norm = vocal / mix_max
    acc_norm = acc / mix_max
    
    if not os.path.exists(f'../DSD100_Npz/{sub}'):
        os.makedirs(f'../DSD100_Npz/{sub}')
    
    print(f"Saving {sample}")
    np.savez_compressed(f'../DSD100_Npz/{sub}/{sample}.npz', mix=mix_norm, vocal=vocal_norm, acc=acc_norm)
    
if __name__ == '__main__':
    if not os.path.exists('../DSD100_Npz'):
        os.makedirs('../DSD100_Npz')
    dirs_dev = list(os.walk('../DSD100_extracted/Dev/'))[0][1]
    # dirs_test = list(os.walk('../DSD100_extracted/Test/'))[0][1]
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs_dev)):
            pool.submit(save_to_npz, '../DSD100_resized/Dev', 'Dev', dirs_dev[i])
        # for i in range(len(dirs_test)):
        #     pool.submit(save_to_npz, '../DSD100_resized/Test', 'Test', dirs_test[i])

Saving 069 - Hollow Ground - Left BlindSaving 054 - Actions - South Of The Water

Saving 065 - Fergessen - Nos Palpitants
Saving 053 - Actions - Devil's Words
Saving 055 - Angels In Amplifiers - I'm Alright
Saving 052 - ANiMAL - Easy Tiger
Saving 078 - Moosmusic - Big Dummy Shake
Saving 077 - Lyndsey Ollard - Catching Up
Saving 068 - Giselle - Moss
Saving 063 - Detsky Sad - Walkie TalkieSaving 064 - Enda Reilly - Cur An Long Ag Seol

Saving 070 - James May - All Souls Moon
Saving 051 - AM Contra - Heart Peripheral
Saving 056 - Arise - Run Run Run
Saving 082 - Punkdisco - Oral Hygiene
Saving 072 - Jay Menon - Through My EyesSaving 081 - Patrick Talbot - Set Me Free

Saving 058 - Ben Carrigan - We'll Talk About It All Tonight
Saving 074 - Juliet's Rescue - HeartbeatsSaving 062 - Cristina Vane - So Easy

Saving 061 - Chris Durban - CelebrateSaving 073 - Johnny Lokke - Whisper To A Scream

Saving 075 - Leaf - Summerghost
Saving 066 - Flags - 54
Saving 071 - James May - If You Say
Saving 05